In [1]:
import math
import os
import re
import requests
import time

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = 'https://avatar.fandom.com/wiki/Avatar_Wiki:Transcripts'
driver_loc = r"C:\Users\Kai Jing\Desktop\KJ's Online Sync\Selenium\chromedriver.exe"

driver = webdriver.Chrome(executable_path=driver_loc)
driver.get(url)

def get_link(i, j, k):
    xpath = '//*[@id="mw-content-text"]/div/table/tbody/tr/td/table['+str(i)+']/tbody/tr/td['+str(j)+']/table/tbody/tr['+str(k)+']/td/a'
    elem = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))
    
    return str(elem.text), elem.get_attribute("href")

links = {}

tree_structure = {2: {1: list(range(1,11)), 2: list(range(1,11))}, 
                  3: {1: list(range(1,11)), 2: list(range(1,11))}, 
                  4: {1: list(range(2,12)), 2: list(range(1,12))}}

for i in tree_structure:
    for j in tree_structure[i]:
        for k in tree_structure[i][j]:
            text, link = get_link(i, j, k)
            links.update({text: link})
            
print(links)

driver.close()

{'The Boy in the Iceberg': 'https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg', 'The Avatar Returns': 'https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns', 'The Southern Air Temple': 'https://avatar.fandom.com/wiki/Transcript:The_Southern_Air_Temple', 'The Warriors of Kyoshi': 'https://avatar.fandom.com/wiki/Transcript:The_Warriors_of_Kyoshi', 'The King of Omashu': 'https://avatar.fandom.com/wiki/Transcript:The_King_of_Omashu', 'Imprisoned': 'https://avatar.fandom.com/wiki/Transcript:Imprisoned', 'Winter Solstice, Part 1: The Spirit World': 'https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_1:_The_Spirit_World', 'Winter Solstice, Part 2: Avatar Roku': 'https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_2:_Avatar_Roku', 'The Waterbending Scroll': 'https://avatar.fandom.com/wiki/Transcript:The_Waterbending_Scroll', 'Jet': 'https://avatar.fandom.com/wiki/Transcript:Jet_(episode)', 'The Great Divide': 'https://avatar.fandom.com/wiki/Trans

In [2]:
episodes = [(i,j) for i in range(1,3) for j in range(1,21)] + [(3,j) for j in range(1,22)]

master_df = pd.DataFrame()

for i, key in enumerate(links):
    url = links[key]
    episode = episodes[i]
    
    page_html = requests.get(url).content
    soup = BeautifulSoup(page_html, 'html.parser')
    
    all_text = [item.text for item in soup.find_all('tr')]
    text_cleaned = [item.replace('\n', '', 2).split('\n')[:2] for item in all_text]
    
    text_container = []
    
    for line in text_cleaned:
        structured_text = {'character': line[0], 'text': line[1]}
        text_container.append(structured_text)
    
    df_text = pd.DataFrame(text_container)
    
    df_text['book'] = episode[0]
    df_text['episode'] = episode[1]
    df_text['title'] = key
    df_text['url'] = url
    
    df_text = pd.concat([df_text.iloc[:,-4:], df_text.iloc[:,:-4]], axis=1)
    
    master_df = pd.concat([master_df, df_text], axis=0)
    
master_df.reset_index(drop=True, inplace=True)
master_df

,book,episode,title,url,character,text
0,1,1,The Boy in the Iceberg,https://avatar.fandom.com/wiki/Transcript:The_...,Katara,Water. Earth. Fire. Air. My grandmother used t...
1,1,1,The Boy in the Iceberg,https://avatar.fandom.com/wiki/Transcript:The_...,,"As the title card fades, the scene opens onto ..."
2,1,1,The Boy in the Iceberg,https://avatar.fandom.com/wiki/Transcript:The_...,Sokka,It's not getting away from me this time. [Clos...
3,1,1,The Boy in the Iceberg,https://avatar.fandom.com/wiki/Transcript:The_...,,"The shot pans quickly from Sokka to Katara, wh..."
4,1,1,The Boy in the Iceberg,https://avatar.fandom.com/wiki/Transcript:The_...,Katara,"[Happily surprised.] Sokka, look!"
...,...,...,...,...,...,...
13443,3,21,"Sozin's Comet, Part 4: Avatar Aang",https://avatar.fandom.com/wiki/Transcript:Sozi...,Sokka,I thought it looked more exciting that way. [M...
13444,3,21,"Sozin's Comet, Part 4: Avatar Aang",https://avatar.fandom.com/wiki/Transcript:Sozi...,Iroh,"[Points at painting.] Hey, my belly's not that..."
13445,3,21,"Sozin's Comet, Part 4: Avatar Aang",https://avatar.fandom.com/wiki/Transcript:Sozi...,Toph,Well I think you all look perfect! [They laugh.]
13446,3,21,"Sozin's Comet, Part 4: Avatar Aang",https://avatar.fandom.com/wiki/Transcript:Sozi...,,"Aang walks past Appa, petting him briefly, bef..."


In [3]:
master_df.to_csv('atla_script.csv', index=False)